In [138]:
# Подключаем нужные пакеты
# Здесь мы будем пользоваться pandas и точками с полигонами из shapely 
import pandas as pd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

,user_id,loc_lat,loc_lon
0,1,55.737564,37.345186
1,2,56.234564,37.234590
2,3,55.234578,36.295745


In [242]:
#pd.read_csv('user_coordinates.csv')
#pd.read_csv('place_zone_coordinates.csv')

# Читаем csv с координамами полигонов и сортируем его по месту и номеру точки, 
# чтобы точки собирались в лист в нужной последовательности.
# Вообще, можно представить ситуацию, в которой пропущена одна из точек полигона и тогда бы хорошо получать какой-нибудь warning
# Но пока оставим так. Вообще строки из примера и без сорта нормально обрабатывались, но на всякий случай
place_zone_coordinates = pd.read_csv('place_zone_coordinates.csv').sort_values(['place_id', 'point_number'])

# Объединяем координаты в кортежи
place_zone_coordinates['loc'] = place_zone_coordinates[['loc_lat', 'loc_lon']].apply(tuple, axis=1)

# Затем группируем кортежи в листы по place_id, листы собираем в полигоны. 
# Благодаря тому, что мы отсортировали данные на этапе загузки, кортежи соединятся в нужной последовательности
# Получаем таблицу, где каждому place_id соответствует полигон, описывающий его зону доставки
place_areas = place_zone_coordinates.groupby(['place_id'])['loc'].apply(list).apply(Polygon).reset_index().rename(columns={'loc':'area'})


# Функция, которая подсчитывает сколько ресторанов доступно в заданной точке
# Точки на границе полигона считаются за доступные
def count_available_places(location_point, area_polygons_series = place_areas['area']):
    place_accessability = []
    for service_area in area_polygons_series:
        place_accessability.append(service_area.covers(location_point))
    return place_accessability.count(True)


# Читаем csv с координатами пользователя
user_coordinates = pd.read_csv('user_coordinates.csv')

# И аналогично объединяем координаты в точки
user_coordinates['user_location'] = user_coordinates[['loc_lat', 'loc_lon']].apply(Point, axis=1)

# Для каждого пользователя применяем обозначенную выше функцию
# Изначально я планировал смёрджить таблицы с пользователями и полигонами и уже в полученной таблице посчитать всё что нужно
# Но там должен будет использоваться мёрдж по ключу 1=1 и я не уверен, насколько хорошо это дело будет масштабироваться
# Поэтому решил сделать вот такой функцией
user_coordinates['number_of_places_available'] = user_coordinates['user_location'].apply(count_available_places, area_polygons_series = place_areas['area'])

# Оставляем только нужные столбцы и переименовываем как требует формат
user_places_available = user_coordinates[['user_id','number_of_places_available']].rename(columns={'user_id':'id'})

user_places_available

# # Ну и давайте csv запишем, почему нет?
# user_places_available.to_csv('user_places_available.csv', index = False)

# Подвохом в этом задании было отсутствие в примерах точек с доступными ресторанами? Это меня напрягло в какой-то момент

,id,number_of_places_available
0,1,0
1,2,0
2,3,0
